# Gadgets Infrastructure Test

This notebook tests the new fault-tolerant gadgets infrastructure:
- **coordinates.py**: N-dimensional coordinate utilities
- **layout.py**: Multi-block layout management
- **scheduling.py**: Parallel gate scheduling

Test pattern follows `decoder_smoke_test.ipynb` with pass/fail counts.

In [3]:
import sys
sys.path.insert(0, '..')

# Test results tracking
passed = 0
failed = 0

def test(name: str, condition: bool) -> bool:
    global passed, failed
    if condition:
        print(f"✅ {name}")
        passed += 1
        return True
    else:
        print(f"❌ {name}")
        failed += 1
        return False

## 1. Coordinates Module Tests

Test N-dimensional coordinate utilities.

In [4]:
from qectostim.gadgets.coordinates import (
    CoordND,
    get_bounding_box,
    compute_bridge_position,
    translate_coords,
    normalize_coord,
)

print("=== Coordinates Module ===\n")

# Test 1: CoordND type
coord_2d: CoordND = (1.0, 2.0)
coord_3d: CoordND = (1.0, 2.0, 3.0)
coord_4d: CoordND = (1.0, 2.0, 3.0, 4.0)
test("CoordND supports 2D", len(coord_2d) == 2)
test("CoordND supports 3D", len(coord_3d) == 3)
test("CoordND supports 4D", len(coord_4d) == 4)

# Test 2: Bounding box
coords = [(0.0, 0.0), (5.0, 3.0), (2.0, 7.0)]
min_corner, max_corner = get_bounding_box(coords)
test("get_bounding_box min corner", min_corner == (0.0, 0.0))
test("get_bounding_box max corner", max_corner == (5.0, 7.0))

# Test 3: 3D bounding box
coords_3d = [(0.0, 0.0, 0.0), (1.0, 2.0, 3.0), (4.0, 1.0, 2.0)]
min_3d, max_3d = get_bounding_box(coords_3d)
test("get_bounding_box 3D min", min_3d == (0.0, 0.0, 0.0))
test("get_bounding_box 3D max", max_3d == (4.0, 2.0, 3.0))

# Test 4: compute_bridge_position
block_a = [(0.0, 0.0), (1.0, 1.0)]
block_b = [(5.0, 5.0), (6.0, 6.0)]
bridge = compute_bridge_position(block_a, block_b, (0.0, 0.0), (0.0, 0.0))
test("compute_bridge_position midpoint", 2.5 <= bridge[0] <= 3.0 and 2.5 <= bridge[1] <= 3.0)

# Test 5: translate_coords
original = [(0.0, 0.0), (1.0, 1.0)]
offset = (10.0, 20.0)
translated = translate_coords(original, offset)
test("translate_coords applies offset", translated == [(10.0, 20.0), (11.0, 21.0)])

# Test 6: normalize_coord (pad to higher dimension)
coord_short = (1.0, 2.0)
padded = normalize_coord(coord_short, 4)
test("normalize_coord extends to 4D", padded == (1.0, 2.0, 0.0, 0.0))

=== Coordinates Module ===

✅ CoordND supports 2D
✅ CoordND supports 3D
✅ CoordND supports 4D
✅ get_bounding_box min corner
✅ get_bounding_box max corner
✅ get_bounding_box 3D min
✅ get_bounding_box 3D max
✅ compute_bridge_position midpoint
✅ translate_coords applies offset
✅ normalize_coord extends to 4D


True

## 2. Layout Module Tests

Test multi-block layout management.

In [6]:
from qectostim.gadgets.layout import (
    BlockInfo,
    BridgeAncilla,
    QubitIndexMap,
    GadgetLayout,
)

print("=== Layout Module ===\n")

# Create a mock code class for testing
class MockCode:
    """Simple mock code for testing layout."""
    def __init__(self, n_data: int, n_x_anc: int = 0, n_z_anc: int = 0):
        self.n = n_data
        self._n_x_anc = n_x_anc
        self._n_z_anc = n_z_anc
        # Create mock coordinates
        self._data_coords = [(float(i), 0.0) for i in range(n_data)]
    
    @property
    def data_coords(self):
        return self._data_coords
    
    @property
    def x_stabilizer_ancillas(self):
        return list(range(self._n_x_anc))
    
    @property
    def z_stabilizer_ancillas(self):
        return list(range(self._n_z_anc))

# Test 7: Create GadgetLayout
layout = GadgetLayout(dim=2)
test("GadgetLayout creation", layout.dim == 2)

# Test 8: Add block
code1 = MockCode(n_data=5, n_x_anc=2, n_z_anc=2)
block_info = layout.add_block("block_a", code1, offset=(0.0, 0.0))
test("add_block returns BlockInfo", isinstance(block_info, BlockInfo))
test("add_block registers block", "block_a" in layout.blocks)
test("BlockInfo has data_qubit_range", block_info.data_qubit_range is not None and len(block_info.data_qubit_range) == 5)

# Test 9: Add second block with offset
code2 = MockCode(n_data=7, n_x_anc=3, n_z_anc=3)
block_info2 = layout.add_block("block_b", code2, offset=(10.0, 0.0))
test("add_block second block", len(block_info2.data_qubit_range) == 7)
# Check that data ranges don't overlap
test("qubit indices don't overlap", 
     block_info.data_qubit_range.stop <= block_info2.data_qubit_range.start or 
     block_info2.data_qubit_range.stop <= block_info.data_qubit_range.start)

# Test 10: Add bridge ancilla
bridge_idx = layout.add_bridge_ancilla(
    coord=(5.0, 0.0),
    purpose="joint_ZZ",
    connected_blocks=["block_a", "block_b"]
)
test("add_bridge_ancilla returns index", bridge_idx >= 0)
test("bridge_ancilla stored", len(layout.bridge_ancillas) == 1)

# Test 11: Get global coord
coord_a = layout.get_global_coord(list(block_info.data_qubit_range)[0])
test("get_global_coord returns coordinate", coord_a is not None and len(coord_a) == 2)

# Test 12: QubitIndexMap
test("qubit_map has global_coords", len(layout.qubit_map.global_coords) > 0)

=== Layout Module ===

✅ GadgetLayout creation
✅ add_block returns BlockInfo
✅ add_block registers block


AttributeError: 'BlockInfo' object has no attribute 'data_range'

## 3. Scheduling Module Tests

Test parallel gate scheduling with geometric and graph coloring strategies.

In [ ]:
from qectostim.gadgets.scheduling import (
    GateType,
    ScheduledGate,
    CircuitLayer,
    GadgetScheduler,
    merge_schedules,
)

print("=== Scheduling Module ===\n")

# Test 13: GateType enum
test("GateType has CX", GateType.CX.value == "CX")
test("GateType has H", GateType.H.value == "H")

# Test 14: ScheduledGate
gate = ScheduledGate(GateType.CX, (0, 1))
test("ScheduledGate creation", gate.gate_type == GateType.CX)
test("ScheduledGate targets", gate.targets == (0, 1))

# Test 15: CircuitLayer
layer = CircuitLayer(time=0.0)
layer.gates.append(gate)
test("CircuitLayer can add gate", len(layer.gates) == 1)
test("CircuitLayer.get_all_qubits", layer.get_all_qubits() == {0, 1})

# Test 16: CircuitLayer conflict detection
gate2 = ScheduledGate(GateType.CX, (1, 2))  # Conflicts with gate (shares qubit 1)
gate3 = ScheduledGate(GateType.CX, (3, 4))  # No conflict
test("CircuitLayer detects conflict", not layer.can_add_gate(gate2))
test("CircuitLayer allows non-conflict", layer.can_add_gate(gate3))

# Test 17: GadgetScheduler creation
scheduler = GadgetScheduler(layout, time_start=0.0)
test("GadgetScheduler creation", scheduler.current_time == 0.0)

In [ ]:
print("=== Graph Coloring Schedule ===\n")

# Test 18: Graph coloring for parallel CNOTs
scheduler2 = GadgetScheduler(layout, time_start=0.0)

# Create CNOT pairs that have various conflicts
# Pairs: (0,1), (2,3), (1,2), (4,5)
# (0,1) conflicts with (1,2)
# (2,3) conflicts with (1,2)
# (4,5) has no conflicts
pairs = [(0, 1), (2, 3), (1, 2), (4, 5)]
scheduler2.schedule_two_qubit_gates_geometric(GateType.CX, pairs, schedule=None)

# Should require at least 2 layers due to conflicts
test("Graph coloring creates layers", scheduler2.get_circuit_depth() >= 2)

# Check no layer has conflicting gates
for layer in scheduler2.layers:
    qubits_used = set()
    conflict_found = False
    for gate in layer.gates:
        if any(q in qubits_used for q in gate.targets):
            conflict_found = True
            break
        qubits_used.update(gate.targets)
    test(f"Layer {layer.time}: no conflicts", not conflict_found)

# Test 19: Statistics
stats = scheduler2.get_stats()
test("Stats tracks two_qubit_gates", stats["two_qubit_gates"] == 4)
print(f"\nSchedule stats: {stats}")

In [ ]:
print("=== Geometric Schedule ===\n")

# Test 20: Geometric schedule (pre-computed)
scheduler3 = GadgetScheduler(layout, time_start=0.0)

# Pre-computed schedule: layer 0 has pairs 0,1,3; layer 1 has pair 2
pairs = [(0, 1), (2, 3), (1, 2), (4, 5)]
geometric_schedule = [[0, 1, 3], [2]]  # Pair indices per layer

scheduler3.schedule_two_qubit_gates_geometric(GateType.CX, pairs, schedule=geometric_schedule)
test("Geometric schedule uses 2 layers", scheduler3.get_circuit_depth() == 2)

# Test 21: Generate Stim instructions
instructions = scheduler3.to_stim_instructions()
test("to_stim_instructions generates output", len(instructions) > 0)
print(f"\nStim instructions preview:")
for instr in instructions[:6]:
    print(f"  {instr}")

In [ ]:
print("=== Full Gadget Workflow ===\n")

# Test 22: Complete stabilizer round scheduling
scheduler4 = GadgetScheduler(layout, time_start=0.0)

# Simulate a stabilizer round
data_qubits = [0, 1, 2, 3]
ancilla_qubits = [10, 11]
cnot_pairs = [(10, 0), (10, 1), (11, 2), (11, 3)]

final_time = scheduler4.schedule_stabilizer_round(
    block_name="test_block",
    stabilizer_type="Z",
    data_qubits=data_qubits,
    ancilla_qubits=ancilla_qubits,
    cnot_pairs=cnot_pairs,
    cnot_schedule=None  # Use graph coloring
)

test("schedule_stabilizer_round completes", final_time > 0)
test("schedule_stabilizer_round includes measurements", scheduler4.get_stats()["measurements"] == 2)
test("schedule_stabilizer_round includes resets", scheduler4.get_stats()["resets"] == 2)

print(f"\nStabilizer round stats: {scheduler4.get_stats()}")

In [ ]:
print("=== Transversal Gate Scheduling ===\n")

# Test 23: Transversal single-qubit gate
scheduler5 = GadgetScheduler(layout, time_start=0.0)
block_data = list(range(5))
scheduler5.schedule_transversal_gate(GateType.H, block_data)
test("schedule_transversal_gate adds gates", scheduler5.get_stats()["single_qubit_gates"] == 5)

# Test 24: Transversal two-qubit gate between blocks
scheduler6 = GadgetScheduler(layout, time_start=0.0)
control_qubits = [0, 1, 2, 3, 4]
target_qubits = [5, 6, 7, 8, 9]
scheduler6.schedule_transversal_two_qubit(GateType.CX, control_qubits, target_qubits)
test("schedule_transversal_two_qubit adds pairs", scheduler6.get_stats()["two_qubit_gates"] == 5)

In [ ]:
print("=== Joint Measurement Scheduling ===\n")

# Test 25: Joint ZZ measurement
scheduler7 = GadgetScheduler(layout, time_start=0.0)
bridge_qubit = 20
target_qubits = [0, 5]  # One qubit from each block
scheduler7.schedule_joint_measurement(
    bridge_ancilla=bridge_qubit,
    target_qubits=target_qubits,
    measurement_type="ZZ"
)
stats7 = scheduler7.get_stats()
test("schedule_joint_measurement ZZ includes reset", stats7["resets"] == 1)
test("schedule_joint_measurement ZZ includes measurement", stats7["measurements"] == 1)
test("schedule_joint_measurement ZZ includes CNOTs", stats7["two_qubit_gates"] == 2)

# Test 26: Joint XX measurement
scheduler8 = GadgetScheduler(layout, time_start=0.0)
scheduler8.schedule_joint_measurement(
    bridge_ancilla=bridge_qubit,
    target_qubits=target_qubits,
    measurement_type="XX"
)
stats8 = scheduler8.get_stats()
test("schedule_joint_measurement XX includes Hadamards", stats8["single_qubit_gates"] == 2)

In [ ]:
print("=== Merge Schedules ===\n")

# Test 27: Merge multiple schedules
sched_a = GadgetScheduler(layout, time_start=0.0)
sched_a.schedule_transversal_gate(GateType.H, [0, 1, 2])

sched_b = GadgetScheduler(layout, time_start=0.0)
sched_b.schedule_transversal_gate(GateType.X, [3, 4, 5])

merged = merge_schedules([sched_a, sched_b], time_gap=1.0)
test("merge_schedules combines layers", merged.get_circuit_depth() >= 2)
test("merge_schedules preserves gates", merged.get_stats()["single_qubit_gates"] == 6)

## 4. Test Summary

In [ ]:
print("\n" + "=" * 50)
print("GADGETS INFRASTRUCTURE TEST SUMMARY")
print("=" * 50)
print(f"Passed: {passed}")
print(f"Failed: {failed}")
print(f"Total:  {passed + failed}")
print("=" * 50)

if failed == 0:
    print("✅ All tests passed!")
else:
    print(f"❌ {failed} test(s) failed")